# Configurine bia-bob's _behaviour_ through system messages

In this notebook we demonstrate how you can configure what kind of Python code bia-bob will generate. Note that the following configuration will be stored across sessions. When installing a new version of bia-bob, these settings are reset. You can also manually reset bia-bob's configuration by deleting the `.cache\bia-bob` directory in your home directory.

In [1]:
from bia_bob import bob, DEFAULT_SYSTEM_PROMPT

bob.__version__

'0.26.0'

## Default behaviour
Per default, bia-bob will write Python code using specific bio-image analysis Python libraries. That's because its default system messages configures this _behaviour_:

In [2]:
print(DEFAULT_SYSTEM_PROMPT[:640])


You are a extremely talented bioimage analyst and you use Python to solve your tasks unless stated otherwise.
If there are several ways to solve the task, chose the option with the least amount of code.    

## Python specific instructions

When writing python code, you can only use those libraries: {libraries}.
If you create images, show the results and save them in variables for later reuse.
{reusable_variables}
NEVER overwrite the values of the variables and functions that are available.

## Python specific code snippets

If the user asks for those simple tasks, use these code snippets.

{additional_snippets}
{builtin_snippets}



Note that libraries, reusable variables and snippets will be injected when bia-bob is invoked. You can use these placeholders in your custom system prompts, too.

In [3]:
%bob print hello world

In [ ]:
tervehdys = "Hello world"
print(tervehdys)
maailman_tervehdys = "Hello world"
print(maailman_tervehdys)

## Custom system messages
You can configure your own system message. In the following example we enforce bob to always just answer a given question and never write code.

In [4]:
bob.initialize(system_prompt="""
    You are an excellent Python programmer who always 
    uses Finnish variable names in their code.
    """)

In [5]:
%bob Write a for-loop that prints the numbers from 0 to 10.

In [ ]:
for luku in range(11):
    print(luku)

## Default system message
You can reset the config to the default system message like this:

In [6]:
bob.initialize(system_prompt=DEFAULT_SYSTEM_PROMPT)

In [7]:
%bob print hello world

In [ ]:
print("hello world")

## Short Python system message
If you work with small, local, open-weight language models, e.g. via [ollama](https://ollama.com/) it might make sense to provide very short system messages:

In [8]:
bob.initialize(system_prompt="""
    Write Python code for the following task and ensure that the functionality is embedded in a function. 
    Also give an example how to call the function.
    """)

In [9]:
%bob print hello world

In [ ]:
def print_hello_world():
    print("Hello World")

# Calling the function
print_hello_world()

## Domain specific system messages
You can instruct bia-bob to use common Python libraries from specific domains.

In [10]:
from bia_bob import bob
bob.initialize(system_prompt="""
    You are an excellent astronomer and Python programmer.
    You typically use Python libraris from this domain.
""")

In [11]:
%bob Write a function that can detect stars in an image.

In [ ]:
from astropy.io import fits
from photutils import DAOStarFinder
from astropy.stats import sigma_clipped_stats
import numpy as np
import matplotlib.pyplot as plt
from photutils.aperture import CircularAperture

def detect_stars(image_data, fwhm=3.0, threshold=5.0):
    """
    Detect stars in an astronomical image using DAOStarFinder.
    
    Parameters:
    -----------
    image_data : numpy.ndarray
        2D numpy array containing the image data
    fwhm : float, optional
        Full-width at half-maximum (FWHM) of the PSF in pixels
    threshold : float, optional
        Detection threshold in sigma units
    
    Returns:
    --------
    sources : Table
        Table containing the detected sources with their properties
    """
    try:
        # Make sure the image is 2D
        if len(image_data.shape) != 2:
            raise ValueError("Input image must be 2D")

        # Calculate background statistics using sigma-clipped stats
        mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
        
        # Create a DAOStarFinder object
        daofind = DAOStarFinder(fwhm=fwhm, 
                               threshold=threshold*std,
                               sharplo=0.2,
                               sharphi=1.0,
                               roundlo=-1.0,
                               roundhi=1.0)
        
        # Find stars
        sources = daofind(image_data - median)
        
        if sources is None:
            print("No sources were found in the image.")
            return None
            
        print(f"Found {len(sources)} sources.")
        return sources
        
    except Exception as e:
        print(f"Error detecting stars: {str(e)}")
        return None

def plot_detected_stars(image_data, sources):
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    apertures = CircularAperture(positions, r=4.)
    
    plt.imshow(image_data, cmap='gray', origin='lower')
    apertures.plot(color='red', lw=1.5, alpha=0.5)
    plt.show()

def filter_sources(sources, min_flux=None, max_flux=None):
    mask = np.ones(len(sources), dtype=bool)
    if min_flux is not None:
        mask &= sources['flux'] >= min_flux
    if max_flux is not None:
        mask &= sources['flux'] <= max_flux
    return sources[mask]

# Example usage:
if __name__ == "__main__":
    # Example of how to use the function with a FITS file
    try:
        # Load a FITS image
        hdul = fits.open('example_image.fits')
        image_data = hdul[0].data
        
        # Detect stars
        sources = detect_stars(image_data)
        
        if sources is not None:
            # Print the first few detected sources
            print("\nFirst few detected sources:")
            print(sources[:5])
            
            # Access coordinates of detected stars
            x_centroids = sources['xcentroid']
            y_centroids = sources['ycentroid']
            
    except FileNotFoundError:
        print("Example FITS file not found.")
    except Exception as e:
        print(f"Error: {str(e)}")

# For a numpy array
sources = detect_stars(your_image_array)

# For a FITS file
image_data = fits.getdata('your_image.fits')
sources = detect_stars(image_data)

## Terrible system messages
You can also instruct bia-bob to do stupid things. Be careful with this functionality.

In [12]:
bob.initialize(system_prompt="""
    You always write super complicated, hard-to-read python code 
    with a lot of unnecessary loops and weird variable names. 
    Make sure it consists of at least 5 lines of code.
""")

In [13]:
%bob print hello world

Here's an unnecessarily complicated way to print "hello world":

def _mysterioso_func(x_374):
    return ''.join([chr((ord(i) + 3) - 3) for i in x_374])

mysterious_list_of_chars = [104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]
temp_storage_container = []
for enigmatic_value in mysterious_list_of_chars:
    temp_storage_container.append(chr(enigmatic_value))
final_output_string = _mysterioso_func(''.join(temp_storage_container))
print(final_output_string)

In [14]:
# Reset default behaviour, as it would be stored permanently otherwise.
bob.initialize(system_prompt=DEFAULT_SYSTEM_PROMPT)